In [17]:
import torch
import torch.nn as nn
print("CUDA available: ", torch.cuda.is_available())
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import warnings
warnings.filterwarnings("ignore")

import os
current_directory = os.getcwd()
print(current_directory)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=True)


%load_ext autoreload
%autoreload 2

#train_df_dirty = pd.read_csv('/content/drive/MyDrive/Hate/DataSet/train_E6oV3lV.csv', index_col=0)
train_df_dirty = pd.read_csv('./DataSet/train_E6oV3lV.csv', index_col=0)
print(train_df_dirty)
#test_df_dirty = pd.read_csv('/content/drive/MyDrive/Hate/DataSet/test_tweets_anuFYb8.csv', index_col=0)
test_df_dirty = pd.read_csv('./DataSet/test_tweets_anuFYb8.csv', index_col=0)
print(test_df_dirty)

hate_2 = pd.read_csv('./DataSet/labeled_data.csv', index_col=0)
hate_2 = hate_2[hate_2.hate_speech == 1][['hate_speech', 'tweet']].rename(columns={'hate_speech' : 'label'})
print(hate_2)

CUDA available:  True
/home/lucy/Documents/MachineLearning/Glacier_git
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
       label                                              tweet
id                                                             
1          0   @user when a father is dysfunctional and is s...
2          0  @user @user thanks for #lyft credit i can't us...
3          0                                bihday your majesty
4          0  #model   i love u take with u all the time in ...
5          0             factsguide: society now    #motivation
...      ...                                                ...
31958      0  ate @user isz that youuu?ðððððð...
31959      0    to see nina turner on the airwaves trying to...
31960      0  listening to sad songs on a monday morning otw...
31961      1  @user #sikh #temple vandalised in in #calgary,...
31962      0                   thank you @user for you follow  

[31962 r

In [18]:
# List of swearing words provided by the user
swearing_words = [
    "alabama hot pocket", "alaskan pipeline", "anal", "anilingus", "anus", "apeshit", "arsehole",
    "ass", "asshole", "assmunch", "auto erotic", "autoerotic", "babeland", "baby batter", "baby juice",
    "ball gag", "ball gravy", "ball kicking", "ball licking", "ball sack", "ball sucking", "bangbros",
    "bareback", "barely legal", "barenaked", "bastard", "bastardo", "bastinado", "bbw", "bdsm", "beaner",
    "beaners", "beaver cleaver", "beaver lips", "bestiality", "big black", "big breasts", "big knockers",
    "big tits", "bimbos", "birdlock", "bitch", "bitches", "black cock", "blonde action", "blonde on blonde action",
    "blowjob", "blow job", "blow your load", "blue waffle", "blumpkin", "bollocks", "bondage", "boner", "boob",
    "boobs", "booty call", "brown showers", "brunette action", "bukkake", "bulldyke", "bullet vibe", "bullshit",
    "bung hole", "bunghole", "busty", "butt", "buttcheeks", "butthole", "camel toe", "camgirl", "camslut", "camwhore",
    "carpet muncher", "carpetmuncher", "chocolate rosebuds", "circlejerk", "cleveland steamer", "clit", "clitoris",
    "clover clamps", "clusterfuck", "cock", "cocks", "coprolagnia", "coprophilia", "cornhole", "coon", "coons",
    "creampie", "cum", "cumming", "cunnilingus", "cunt", "darkie", "date rape", "daterape", "deep throat", "deepthroat",
    "dendrophilia", "dick", "dildo", "dingleberry", "dingleberries", "dirty pillows", "dirty sanchez", "doggie style",
    "doggiestyle", "doggy style", "doggystyle", "dog style", "dolcett", "domination", "dominatrix", "dommes", "donkey punch",
    "double dong", "double penetration", "dp action", "dry hump", "dvda", "eat my ass", "ecchi", "ejaculation", "erotic",
    "erotism", "escort", "eunuch", "faggot", "fecal", "felch", "fellatio", "feltch", "female squirting", "femdom", "figging",
    "fingerbang", "fingering", "fisting", "foot fetish", "footjob", "frotting", "fuck", "fuck buttons", "fuckin", "fucking",
    "fucktards", "fudge packer", "fudgepacker", "futanari", "gang bang", "gay sex", "genitals", "giant cock", "girl on",
    "girl on top", "girls gone wild", "goatcx", "goatse", "god damn", "gokkun", "golden shower", "goodpoop", "goo girl",
    "goregasm", "grope", "group sex", "g-spot", "guro", "hand job", "handjob", "hard core", "hardcore", "hentai", "homoerotic",
    "honkey", "hooker", "hot carl", "hot chick", "how to kill", "how to murder", "huge fat", "humping", "incest", "intercourse",
    "jack off", "jail bait", "jailbait", "jelly donut", "jerk off", "jigaboo", "jiggaboo", "jiggerboo", "jizz", "juggs", "kike",
    "kinbaku", "kinkster", "kinky", "knobbing", "leather restraint", "leather straight jacket", "lemon party", "lolita",
    "lovemaking", "make me come", "male squirting", "masturbate", "menage a trois", "milf", "missionary position", "motherfucker",
    "mound of venus", "mr hands", "muff diver", "muffdiving", "nambla", "nawashi", "negro", "neonazi", "nigga", "nigger", "nig nog",
    "nimphomania", "nipple", "nipples", "nsfw images", "nude", "nudity", "nympho", "nymphomania", "octopussy", "omorashi", "one cup two girls",
    "one guy one jar", "orgasm", "orgy", "paedophile", "paki", "panties", "panty", "pedobear", "pedophile", "pegging", "penis", "phone sex",
    "piece of shit", "pissing", "piss pig", "pisspig", "playboy", "pleasure chest", "pole smoker", "ponyplay", "poof", "poon", "poontang",
    "punany", "poop chute", "poopchute", "porn", "porno", "pornography", "prince albert piercing", "pthc", "pubes", "pussy", "queaf", "queef",
    "quim", "raghead", "raging boner", "rape", "raping", "rapist", "rectum", "reverse cowgirl", "rimjob", "rimming", "rosy palm", "rosy palm and her 5 sisters",
    "rusty trombone", "sadism", "santorum", "scat", "schlong", "scissoring", "semen", "sex", "sexo", "sexy", "shaved beaver", "shaved pussy", "shemale",
    "shibari", "shit", "shitblimp", "shitty", "shota", "shrimping", "skeet", "slanteye", "slut", "s&m", "smut", "snatch", "snowballing", "sodomize", "sodomy",
    "spic", "splooge", "splooge moose", "spooge", "spread legs", "spunk", "strap on", "strapon", "strappado", "strip club", "style doggy", "suck", "sucks",
    "suicide girls", "sultry women", "swastika", "swinger", "tainted love", "taste my", "tea bagging", "threesome", "throating", "tied up", "tight white", "tit",
    "tits", "titties", "titty", "tongue in a", "topless", "tosser", "towelhead", "tranny", "tribadism", "tub girl", "tubgirl", "tushy", "twat", "twink", "twinkie",
    "two girls one cup", "undressing", "upskirt", "urethra play", "urophilia", "vagina", "venus mound", "vibrator", "violet wand", "vorarephilia", "voyeur",
    "vulva", "wank", "wetback", "wet dream", "white power", "wrapping men", "wrinkled starfish", "xx", "xxx", "yaoi", "yellow showers", "yiffy", "zoophilia"
]

misogynistic_words = [
    "harpy", "nag", "scold", "shrew", "slattern", "siren", "temptress",
    "termagant", "trollop", "vixen", "wench", "witch"
]

incel_words = [
    "beta orbiter", "black pill", "celibate", "coomer", "gymcel", "looksmax",
    "neet", "truecel", "volcel", "wagecel"
]

race_abusive_words = [
    "blackamoor", "boong", "burrhead", "chug", "coolie", "dinge", "dothead",
    "golliwog", "jig", "kaffir", "mud person", "mud shark", "moulinyan",
    "mutt", "octoroon", "papoose", "pickaninny", "raghead", "sambo",
    "skinhead", "snowflake", "spearchucker", "squaw", "tar baby", "whitey"
]

transgender_abusive_words = [
    "cis scum", "gender bender", "gender freak", "it", "ladyboy", "shim",
    "tg", "tranny", "trap"
]

muslim_abusive_words = [
    "baghead", "bomber", "camel fucker", "goat lover", "momo", "muslim scum",
    "raghead", "towelhead"
]

sexist_abusive_words = [
    "broad", "doll", "honey", "hostess", "moll", "skirt", "wench"
]

abusive_lefties = [
    "libtard", "snowflake", "leftard", "commie", "pinko", "treehugger",
    "feminazi", "libturd", "leftie", "left-wing nutjob"
]

trumpists_abusive = [
    "cuck", "fake news", "globalist", "liberal scum", "snowflake", "traitor",
    "libtard", "deep state", "crooked Hillary", "nasty woman"
]

trumpist_abusive_expressions = [
    "libtard", "snowflake", "leftard", "commie", "pinko", "socialist scum", "marxist loser",
    "radical leftist", "fake news", "media liar", "mainstream media shill", "antifa thug",
    "left-wing extremist", "democrat clown", "crooked Hillary", "nasty woman", "deep state agent",
    "traitor", "America hater", "unpatriotic scum", "blue state loser", "leftist propaganda peddler",
    "anarchist rioter", "black lives matter terrorist", "climate change hoaxer", "feminazi",
    "radical feminist", "welfare queen", "open borders supporter", "illegal immigrant lover",
    "gun grabber", "religious persecutor", "christian hater", "big government advocate",
    "tax and spend liberal", "green new deal freak", "environmental nutjob", "covid hoax believer",
    "mask nazi", "vaccine pusher", "cancel culture warrior", "identity politics fanatic",
    "race baiter", "diversity pusher", "woke moron", "gender bender", "transgender agenda promoter",
    "school indoctrinator", "CRT peddler", "defund the police supporter", "social justice warrior",
    "occupy wall street radical", "me too movement nutcase", "hollywood elite", "celebrity hypocrite",
    "china lover", "russia collusion liar", "impeachment fanatic", "nancy pelosi puppet",
    "schumer shill", "progressive puppet", "squad member", "AOC fanboy", "bernie bro",
    "leftist mob", "resistance member", "never-trumper", "rino", "establishment hack",
    "globalist", "george soros minion", "deep state operative", "fake refugee supporter",
    "traitorous diplomat", "election fraud conspirator", "unhinged protester", "mob rule advocate",
    "portland anarchist", "seattle autonomous zone supporter", "sanctuary city lover",
    "hollywood liberal", "silicon valley censor", "big tech tyrant", "liberal academic",
    "ivy league snob", "antifa sympathizer", "progressive prosecutor", "defund ICE supporter",
    "socialist healthcare pusher", "medicare for all supporter", "green energy nut",
    "paris accord supporter", "Iran deal lover", "North Korea appeaser", "china appeaser",
    "trade deal critic", "military budget cutter", "foreign aid lover", "open borders activist",
    "Islamic terrorist sympathizer", "radical imam supporter", "Biden bot", "Kamala puppet"
]

xenophobic_abusive_expressions = [
    "illegal alien", "invader", "parasite", "job stealer", "freeloader", "anchor baby", 
    "welfare leech", "economic migrant", "terrorist", "criminal alien", "refugee scum", 
    "border hopper", "drug dealer", "gang member", "disease spreader", "dirty foreigner", 
    "third-world invader", "illegal immigrant", "wetback", "beaner", "spic", "chink", 
    "gook", "raghead", "towelhead", "sand nigger", "camel jockey", "bomb maker", "jihadi", 
    "mujahid", "infidel", "kafir", "illegal worker", "visa overstayer", "undocumented", 
    "non-citizen", "outsider", "foreigner", "alien", "non-native", "intruder", "trespasser", 
    "expat", "transient", "displaced person", "economic refugee", "guest worker", "asylum seeker", 
    "naturalized citizen", "first-generation", "second-generation", "ethnic minority", 
    "newcomer", "settler", "migrant laborer", "foreign national", "stateless person", 
    "illegal entrant", "cross-border", "immigrant", "new arrival", "migrant", "visa holder", 
    "ethnic invader", "race mixer", "mongrel", "half-breed", "mixed blood", "blood traitor", 
    "race traitor", "white genocide promoter", "racial inferior", "genetic inferior", 
    "subhuman", "non-Aryan", "race criminal", "race defiler", "unassimilated", "unintegrated", 
    "tribalist", "ethnic bloc", "ethnic enclave", "foreign colony", "diaspora", "cross-cultural", 
    "multi-ethnic", "multicultural", "polyethnic", "interracial", "race-blind", "color-blind", 
    "diversity proponent", "race equalizer", "race leveler", "race integrator", "race harmonizer"
]

trumpist_evangelical_abusive_expressions = [
    "godless liberal", "satanic democrat", "immoral atheist", "secular humanist", "anti-Christian bigot",
    "leftist heathen", "god-hater", "Christ-hater", "Bible-basher", "false prophet", "corrupt sinner",
    "liberal apostate", "Christ-denier", "abortionist", "baby killer", "pro-choice murderer", "anti-life scum",
    "gay agenda pusher", "LGBTQ radical", "homosexual deviant", "transgender pervert", "gender confusion promoter",
    "drag queen supporter", "family destroyer", "anti-family activist", "marriage destroyer", "adulterer", 
    "fornicator", "pedophile enabler", "child abuser", "godless commie", "communist infiltrator", 
    "socialist agitator", "religious freedom hater", "church attacker", "worship suppressor", 
    "biblical values hater", "Christian values enemy", "cross-burner", "bible-burner", "godless educator", 
    "atheist teacher", "indoctrinator", "public school atheist", "university atheist", "godless professor", 
    "secular scientist", "evolution pusher", "climate change fanatic", "tree hugger", "eco-freak", 
    "godless entertainer", "Hollywood heathen", "celebrity atheist", "music industry satanist", 
    "anti-christian activist", "court god-hater", "judge atheist", "lawyer satanist", "anti-prayer warrior", 
    "school prayer hater", "public square atheist", "freedom from religion pusher", "godless bureaucrat", 
    "government atheist", "deep state atheist", "federal god-hater", "local government atheist", 
    "state government atheist", "media god-hater", "fake news atheist", "mainstream media satanist", 
    "liberal journalist", "godless reporter", "secular broadcaster", "anti-Christian media", 
    "abortion clinic worker", "Planned Parenthood supporter", "sex education pusher", "condom distributor", 
    "gay marriage supporter", "same-sex marriage advocate", "gender-neutral bathroom supporter", 
    "drag queen story hour promoter", "godless politician", "godless congressman", "secular senator", 
    "atheist president", "anti-Christian governor", "godless mayor", "godless city council member", 
    "leftist church leader", "liberal pastor", "godless bishop", "satanic preacher", "anti-christian reverend"
]

anti_feminist_abusive_expressions = [
    "feminazi", "man-hater", "femscum", "gender terrorist", "militant feminist", "radical feminist",
    "feminist whacko", "feminist nutjob", "feminist extremist", "men's rights hater", "women's libber",
    "bra burner", "angry feminist", "toxic feminist", "feminist zealot", "feminist bigot", "feminist pig",
    "man basher", "female supremacist", "misandrist", "gender bender", "liberal feminist", "intersectional feminist",
    "third-wave feminist", "fourth-wave feminist", "feminist propagandist", "equality freak", "gender radical",
    "female chauvinist", "gender equality extremist", "PC feminist", "progressive feminist", "women's lib extremist",
    "women's rights radical", "feminist dictator", "gender justice warrior", "SJW feminist", "PC police feminist",
    "feminist overlord", "man-hating feminist", "patriarchy smasher", "male basher", "anti-male feminist",
    "male oppressor", "male privilege denier", "gender warrior", "sexist feminist", "gender equity freak",
    "feminist hypocrite", "misandrist feminist", "women's rights fanatic", "gender revolutionist", "feminist agitator",
    "feminist crusader", "feminist activist", "gender equity warrior", "feminist insurgent", "feminist troublemaker",
    "feminist rabble-rouser", "feminist militant", "feminist extremist", "feminist terrorist", "gender radicalizer",
    "gender equality zealot", "feminist ideologue", "feminist hardliner", "feminist fanatic", "feminist hardhead",
    "feminist brainwasher", "gender equality dictator", "feminist regime supporter", "feminist dictator",
    "feminist enforcer", "feminist totalitarian", "gender equality extremist", "feminist fundamentalist",
    "feminist crusader", "feminist warrior", "feminist insurgent", "feminist zealot", "feminist authoritarian",
    "feminist radicalizer", "gender equity extremist", "gender equality fanatic", "feminist autocrat",
    "gender equity zealot", "feminist revolutionary", "feminist oppressor", "feminist autocrat", "feminist tyrant",
    "gender equality tyrant", "feminist totalitarian", "feminist oppressor", "feminist enforcer", "feminist dictator"
]

anti_lgbt_abusive_expressions = [
    "homophobe", "fag", "faggot", "dyke", "tranny", "queer", "sodomite", "pillow biter",
    "rump ranger", "butt pirate", "queer bait", "nancy boy", "poofter", "fairy", "fruit",
    "pansy", "lezzo", "he-she", "she-male", "gender bender", "fruitcake", "gaylord",
    "homo", "carpet muncher", "muff diver", "lesbo", "fudge packer", "backdoor bandit",
    "sissy", "nancy", "twink", "twinkie", "breeder", "gold star", "pillow queen",
    "queen", "queen bee", "bent", "genderqueer", "genderfreak", "hermaphrodite", "invert",
    "genderfuck", "genderless", "cis scum", "fruit loop", "rainbow warrior", "flamer",
    "pillow biter", "pillow muncher", "rump wrangler", "bum bandit", "arse bandit", 
    "arse pirate", "cocksucker", "boy lover", "ball licker", "rainbow chaser", "lipstick lesbian",
    "homo thug", "fruit bat", "fairy boy", "friend of Dorothy", "bugger", "gender terrorist",
    "lesbian wannabe", "beard", "flit", "twinkie eater", "ladyboy", "genderqueer freak",
    "pansexual", "genderfluid", "cisgender hater", "drag queen", "drag king", "cross-dresser",
    "effeminate", "butch", "stone butch", "lipstick femme", "gender outlaws", "sexual deviant",
    "sexually confused", "mentally ill", "unnatural", "deviant", "sinful", "abomination",
    "pervert", "perverted", "predator", "recruiter", "child molester", "child abuser"
]

combined_set = set(
    swearing_words +
    misogynistic_words +
    incel_words +
    race_abusive_words +
    transgender_abusive_words +
    muslim_abusive_words +
    sexist_abusive_words +
    abusive_lefties +
    trumpists_abusive +
    trumpist_abusive_expressions +
    xenophobic_abusive_expressions +
    trumpist_evangelical_abusive_expressions +
    anti_feminist_abusive_expressions +
    anti_lgbt_abusive_expressions
)

# Convert the set back to a list
abusive_words = list(combined_set)
print(len(abusive_words))

897


In [19]:
# Define templates for generating synthetic hate tweets
templates = [
    "You are such a {word}, can't believe it!",
    "Get lost, you {word}!",
    "People like you are the worst, {word}!",
    "No one likes a {word} like you!",
    "Stay away, {word}!",
    "You are the epitome of a {word}.",
    "I can't stand {word}s like you.",
    "Just go away, {word}.",
    "You disgust me, {word}.",
    "Only a {word} would say something like that.",
    "You're nothing but a {word}.",
    "What a {word} thing to do.",
    "Shut up, {word}.",
    "Nobody cares about a {word} like you.",
    "You're such a {word}, seriously.",
    "Get a life, {word}.",
    "Why are you such a {word}?",
    "I hate {word}s like you.",
    "What kind of {word} are you?",
    "You must be proud to be a {word}.",
    "You {word}, stay away from me.",
    "You're the worst kind of {word}.",
    "It's people like you that give {word}s a bad name.",
    "What a loser, {word}.",
    "Go to hell, {word}.",
    "You are nothing but a {word}.",
    "Pathetic {word}.",
    "Typical {word} behavior.",
    "You make me sick, {word}.",
    "You are the definition of a {word}.",
    "How can you be such a {word}?",
    "Just leave, {word}.",
    "You're a joke, {word}.",
    "Shame on you, {word}.",
    "I can't believe you're a {word}.",
    "Disgusting {word}.",
    "You belong in the trash, {word}.",
    "You are a total {word}.",
    "Why would anyone want to be a {word} like you?",
    "Nobody wants to hear from a {word}.",
    "You're a disgrace, {word}.",
    "Absolutely pathetic, {word}.",
    "Why don't you just disappear, {word}?",
    "You're the reason people hate {word}s.",
    "You should be ashamed, {word}.",
    "What a waste of space, {word}.",
    "How can someone be so much of a {word}?",
    "Do everyone a favor and shut up, {word}.",
    "No one wants a {word} around.",
    "You are absolutely worthless, {word}.",
    "Nobody respects a {word} like you.",
    "You're a waste, {word}.",
    "You're pathetic, {word}.",
    "You're a complete {word}.",
    "You're a disgrace to {word}s everywhere.",
    "Everyone hates a {word} like you.",
    "You make the world a worse place, {word}.",
    "You are the worst kind of {word}.",
    "You should feel bad for being a {word}.",
    "You're just another {word}.",
    "Get out of here, {word}.",
    "You're a terrible person, {word}.",
    "Nobody likes a {word}.",
    "You're the lowest of the low, {word}.",
    "You're the worst, {word}.",
    "You're a waste of breath, {word}.",
    "You're just trash, {word}.",
    "You're garbage, {word}.",
    "You're nothing, {word}.",
    "You're a nobody, {word}.",
    "You're scum, {word}.",
    "You're filth, {word}.",
    "You're a piece of garbage, {word}.",
    "You're a piece of trash, {word}.",
    "You're worthless, {word}.",
    "You're nothing but a {word}.",
    "You're a waste of space, {word}.",
    "You're a waste of time, {word}.",
    "You're a waste of life, {word}.",
    "You're a waste of air, {word}.",
    "You're a waste of skin, {word}.",
    "You're a waste of everything, {word}.",
    "You're a waste, {word}.",
    "You're useless, {word}.",
    "You're a burden, {word}.",
    "You're a disgrace, {word}.",
    "You're a blight, {word}.",
    "You're a stain, {word}.",
    "You're a blemish, {word}.",
    "You're an eyesore, {word}.",
    "You're a blot, {word}.",
    "You're a smudge, {word}.",
    "You're a mark, {word}.",
    "You're a scar, {word}.",
    "You're a wound, {word}.",
    "You're a lesion, {word}.",
    "You're a disease, {word}.",
    "You are a total {word}.",
    "Why would anyone want to associate with a {word} like you?",
    "You're a disgrace to humanity, {word}.",
    "Every time you speak, you prove you're a {word}.",
    "You're the lowest of the low, {word}.",
    "I can't stand looking at a {word} like you.",
    "You make everything worse, {word}.",
    "Everyone hates a {word} like you.",
    "The world would be better without {word}s like you.",
    "You're nothing but a {word}.",
    "Go crawl back into the hole you came from, {word}.",
    "You should be ashamed of yourself, {word}.",
    "What a joke, {word}.",
    "You're a pathetic excuse for a human, {word}.",
    "Nobody needs a {word} like you around.",
    "You're a burden to society, {word}.",
    "What a waste of life, {word}.",
    "No one would miss you, {word}.",
    "You're a stain on society, {word}.",
    "You're an embarrassment, {word}.",
    "People like you make me sick, {word}.",
    "You're a waste of oxygen, {word}.",
    "Why don't you just disappear, {word}?",
    "You're a nuisance, {word}.",
    "Nobody cares about your opinion, {word}.",
    "You're irrelevant, {word}.",
    "What a loser, {word}.",
    "Go away, {word}.",
    "You're an idiot, {word}.",
    "You have no place here, {word}.",
    "You're not welcome here, {word}.",
    "What a disgrace, {word}.",
    "You're an insult to humanity, {word}.",
    "You make everything worse, {word}.",
    "You're a joke, {word}.",
    "You're pathetic, {word}.",
    "You're worthless, {word}.",
    "What a waste of space, {word}.",
    "Nobody likes you, {word}.",
    "You're a failure, {word}.",
    "You're a nobody, {word}.",
    "What a waste, {word}.",
    "You're a blight on society, {word}.",
    "You're a burden, {word}.",
    "What a disgrace, {word}.",
    "You're an eyesore, {word}.",
    "You're a waste of time, {word}.",
    "You're irrelevant, {word}.",
    "Nobody needs you, {word}.",
    "You're a pest, {word}.",
    "You're an annoyance, {word}.",
    "What a waste of air, {word}.",
    "You're a disgrace to your family, {word}.",
    "You're a disappointment, {word}.",
    "Nobody respects you, {word}.",
    "You're a joke to everyone, {word}.",
    "Nobody likes a {word}.",
    "What a pathetic {word}.",
    "You're a disgrace to yourself, {word}.",
    "You're a failure, {word}.",
    "Nobody cares about a {word} like you.",
    "You're a joke to society, {word}.",
    "You're a loser, {word}.",
    "You're a waste of life, {word}.",
    "You're a waste of everything, {word}.",
    "You're useless, {word}.",
    "You're a waste of potential, {word}.",
    "Nobody wants you around, {word}.",
    "You're a burden to everyone, {word}.",
    "You're a total waste, {word}.",
    "You're an absolute failure, {word}.",
    "You're a waste of talent, {word}.",
    "Nobody wants to hear from a {word} like you.",
    "You're a complete joke, {word}.",
    "You're a waste of resources, {word}.",
    "You're a waste of effort, {word}.",
    "You're a waste of attention, {word}.",
    "You're a waste of everything good, {word}.",
    "You're a total embarrassment, {word}.",
    "You're a disgrace to everything, {word}.",
    "You're a total failure, {word}.",
    "Nobody needs a {word} like you.",
    "You're a complete waste, {word}.",
    "You're a waste of everything valuable, {word}.",
    "You're a waste of space and time, {word}.",
    "You're a waste of good air, {word}.",
    "You're a waste of energy, {word}.",
    "You're a waste of existence, {word}.",
    "You're a total waste of everything, {word}.",
    "Nobody cares about a {word} like you.",
    "You're a total waste of potential, {word}.",
    "You're a total waste of life, {word}.",
    "You're a waste of everyone's time, {word}.",
    "You're a waste of everyone's effort, {word}.",
    "You're a waste of everyone's attention, {word}.",
    "You're a total waste of everyone's resources, {word}.",
    "{word}, you are such a disgrace!",
    "Can't believe it, {word}, you are the worst!",
    "Stay away from me, {word}.",
    "{word}, nobody likes you!",
    "What a joke, {word}.",
    "You {word}, must be proud to be a failure.",
    "Only a {word} would say something like that.",
    "Shut up, {word}, nobody cares about your opinion.",
    "You're a joke, {word}.",
    "{word}, you're a waste of space.",
    "Get lost, {word}, you're pathetic.",
    "Why don't you just disappear, {word}?",
    "You are nothing but a {word}.",
    "Pathetic {word}, just leave.",
    "{word}, you're a complete joke.",
    "What a loser, {word}.",
    "{word}, you make me sick.",
    "Disgusting {word}, stay away.",
    "Absolutely pathetic, {word}.",
    "You're the worst, {word}.",
    "{word}, you're a disgrace.",
    "Nobody wants a {word} around.",
    "You should feel bad, {word}.",
    "Go to hell, {word}.",
    "You're worthless, {word}.",
    "What kind of {word} are you?",
    "{word}, you are absolutely worthless.",
    "Do everyone a favor and shut up, {word}.",
    "No one respects a {word} like you.",
    "{word}, you're a waste of time.",
    "You're just another {word}.",
    "{word}, nobody needs you.",
    "Get out of here, {word}.",
    "You're a terrible person, {word}.",
    "{word}, you're an idiot.",
    "You have no place here, {word}.",
    "What a disgrace, {word}.",
    "{word}, you're an insult to humanity.",
    "You make everything worse, {word}.",
    "{word}, you're a joke.",
    "What a waste, {word}.",
    "You're a blight on society, {word}.",
    "What a disgrace, {word}.",
    "{word}, you're an eyesore.",
    "You're a waste of air, {word}.",
    "Nobody respects you, {word}.",
    "{word}, you're a joke to everyone.",
    "What a pathetic {word}.",
    "You're a disgrace to yourself, {word}.",
    "You're a failure, {word}.",
    "{word}, you're a loser.",
    "You're a waste of life, {word}.",
    "{word}, you're a waste of everything.",
    "You're useless, {word}.",
    "Nobody wants you around, {word}.",
    "You're a burden to everyone, {word}.",
    "You're a total waste, {word}.",
    "You're an absolute failure, {word}.",
    "{word}, nobody wants to hear from you.",
    "You're a waste of resources, {word}.",
    "You're a waste of attention, {word}.",
    "You're a total embarrassment, {word}.",
    "{word}, you're a disgrace to everything."
]

# # Combine all the templates (if there are more)
# all_templates = templates  # add more templates if you have

# import random
# # Function to generate synthetic hate tweets using the swearing words and templates
# def generate_hate_tweets(swearing_words, templates, num_tweets=400):
#     generated_tweets = []
#     for _ in range(num_tweets):
#         word = random.choice(swearing_words)
#         template = random.choice(templates)
#         tweet = template.format(word=word)
#         generated_tweets.append(tweet)
#     return generated_tweets

# # Generate hate tweets using the swearing words and templates
# generated_tweets = generate_hate_tweets(swearing_words, all_templates, num_tweets=1000)

# # Create a dataframe for the generated tweets
# generated_df = pd.DataFrame({
#     'label': [1] * len(generated_tweets),
#     'tweet': generated_tweets
# })

# # Display the generated hate tweets
# print(generated_df.head())


In [20]:
#!python -m spacy download en_core_web_sm
import spacy
import random
import pandas as pd
import spacy.cli

# Ensure the SpaCy model is downloaded
spacy.cli.download("en_core_web_sm")

# Load SpaCy English tokenizer, POS tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Function to generate permutations of the template
def permute_template(template, max_permutations=5):
    placeholder = "{word}"
    if placeholder not in template:
        return []
    
    # Replace the placeholder with a temporary token for detection
    doc = nlp(template.replace(placeholder, "___"))
    word_index = [token.i for token in doc if token.text == "___"]
    
    if not word_index:
        return []
    
    word_index = word_index[0]
    
    # Extract positions to insert the word
    safe_positions = [i for i, token in enumerate(doc) if token.pos_ in {"NOUN", "VERB", "ADJ", "ADV", "PUNCT"}]
    
    # Generate permutations
    permutations = []
    for _ in range(max_permutations):
        new_position = random.choice(safe_positions)
        if new_position != word_index:  # Avoid the original position
            tokens = [token.text for token in doc]
            tokens.remove("___")  # Remove the placeholder
            tokens.insert(new_position, "{word}")
            permutations.append(" ".join(tokens))
    return permutations

# Define original templates
templates = templates
# Generate augmented templates
augmented_templates = []
for template in templates:
    permutations = permute_template(template)
    if permutations:
        augmented_templates.extend(permutations)
    else:
        # If no permutations are generated, keep the original template
        augmented_templates.append(template)

# Remove duplicates if any
augmented_templates = list(set(augmented_templates))

# Check if augmented_templates is still empty
if not augmented_templates:
    raise ValueError("No augmented templates generated. Please check the permute_template function.")

# Function to generate synthetic hate tweets using the swearing words and templates
def generate_hate_tweets(swearing_words, templates, num_tweets=400):
    if not templates:
        raise ValueError("Template list is empty. Cannot generate tweets.")
    
    generated_tweets = []
    for _ in range(num_tweets):
        word = random.choice(swearing_words)
        template = random.choice(templates)
        tweet = template.format(word=word)
        generated_tweets.append(tweet)
    return generated_tweets

# Sample swearing words (replace with actual swearing words)
swearing_words = swearing_words

# Generate hate tweets using the swearing words and augmented templates
generated_tweets = generate_hate_tweets(swearing_words, augmented_templates, num_tweets=10000)

# Create a dataframe for the generated tweets
generated_df = pd.DataFrame({
    'label': [1] * len(generated_tweets),
    'tweet': generated_tweets
})


# Display the combined and shuffled DataFrame
print(generated_df.head(10))

   label                                              tweet
0      1                      Disgusting sadism, stay away.
1      1           You are absolutely worthless, tribadism.
2      1              You belong in the trash, nymphomania.
3      1                  What kind of nimphomania are you?
4      1                  You're a waste of talent, raping.
5      1                   Nobody needs a raghead like you.
6      1                           You're a blemish, felch.
7      1            You're a waste of space and time, dick.
8      1         You're a waste of everything good, femdom.
9      1  Every time you speak, you prove you're a date ...


In [21]:
print(len(generated_df))

duplicates = generated_df[generated_df.duplicated(subset='tweet', keep=False)]

# Display the duplicates
print("Duplicates based on 'tweet' column:")
print(duplicates)

10000
Duplicates based on 'tweet' column:
      label                                            tweet
13        1               You're an absolute failure, queaf.
18        1                      mr hands, nobody likes you!
29        1                               Get a life, bitch.
30        1           figging, you are absolutely worthless.
33        1              No one likes a doggystyle like you!
...     ...                                              ...
9962      1    You're a joke to everyone, cleveland steamer.
9973      1                      footjob, you're a disgrace.
9978      1                            You're nothing, suck.
9988      1     You're a pathetic excuse for a human, coons.
9996      1  Every time you speak, you prove you're a skeet.

[1241 rows x 2 columns]


In [22]:
# Remove duplicates
generated_df = generated_df.drop_duplicates(subset='tweet')
duplicates = generated_df[generated_df.duplicated(subset='tweet', keep=False)]

# Display the duplicates
print("Duplicates based on 'tweet' column:")
print(len(duplicates))
print(len(generated_df))

Duplicates based on 'tweet' column:
0
9369


In [23]:
combined_df = pd.concat([train_df_dirty, hate_2, generated_df],ignore_index=True)
train_df_dirty = combined_df.sample(frac=1).reset_index(drop=True)
train_df_dirty 

,label,tweet
0,1,You're a complete penis.
1,0,@user singing makes me ð¶ðð
2,1,"You're a waste of resources, shaved pussy."
3,0,facce da set! ó¾´ #thereaping #thereapingthes...
4,1,"You're a total waste of everything, lolita."
...,...,...
44745,0,@user how is this parking by one member of you...
44746,1,"What a loser, poop chute."
44747,1,&#8220;@thekiddfranky: I talk to any girl who ...
44748,0,this makes me incredibly happy #oitnb #netflix...


In [24]:
train_df_dirty.rename(columns={'index': 'id'}, inplace=True)
train_df_dirty.index.name = 'id'
train_df_dirty

,label,tweet
id,,
0,1,You're a complete penis.
1,0,@user singing makes me ð¶ðð
2,1,"You're a waste of resources, shaved pussy."
3,0,facce da set! ó¾´ #thereaping #thereapingthes...
4,1,"You're a total waste of everything, lolita."
...,...,...
44745,0,@user how is this parking by one member of you...
44746,1,"What a loser, poop chute."
44747,1,&#8220;@thekiddfranky: I talk to any girl who ...


In [25]:
duplicates = train_df_dirty[train_df_dirty.duplicated(subset='tweet', keep=False)]

# Display the duplicates
print("Duplicates based on 'label' column:")
print(duplicates)

Duplicates based on 'label' column:
       label                                              tweet
id                                                             
6          1  ð @user  @user own team up with #new #mc @u...
29         0  weddings are impoant because they celebrate li...
42         0  #model   i love u take with u all the time in ...
51         0   i am prepared. #i_am #positive #affirmation     
59         0  can #lighttherapy help with   or #depression? ...
...      ...                                                ...
44665      0  looking to feel more #joy? join me and 20 spea...
44666      0  #likescam   bull hill climb: you have to reach...
44708      1  @user you might be a libtard if... #libtard  #...
44715      0  i finally found a way how to delete old tweets...
44729      0  find out about making schools   places @user @...

[3126 rows x 2 columns]


In [26]:
# If you want to remove duplicates based on the 'label' column
train_df_dirty_no_duplicates = train_df_dirty.drop_duplicates(subset='tweet')

# Display the DataFrame after removing duplicates
print("DataFrame after removing duplicates:")
print(len(train_df_dirty_no_duplicates))

DataFrame after removing duplicates:
42318


In [29]:
train_df_dirty_no_duplicates.to_csv('./DataSet/train_df_dirty.csv', index=True)
train_df_dirty = pd.read_csv('./DataSet/train_df_dirty.csv', index_col='id')
train_df_dirty

,label,tweet
id,,
0,1,You're a complete penis.
1,0,@user singing makes me ð¶ðð
2,1,"You're a waste of resources, shaved pussy."
3,0,facce da set! ó¾´ #thereaping #thereapingthes...
4,1,"You're a total waste of everything, lolita."
...,...,...
44745,0,@user how is this parking by one member of you...
44746,1,"What a loser, poop chute."
44747,1,&#8220;@thekiddfranky: I talk to any girl who ...


Augmentated as above data set 'train_df_dirty' will be used for subsequent training. Since we use GPT-2 for tokanisation, data normalization and standardization for training a GPT model using Hugging Face's transformers library primarily involve ensuring the text is cleaned and properly tokenized. Since we will be using a tokenizer (GPT2Tokenizer), the primary preprocessing steps are to ensure the text is clean and ready for tokenization.

In [30]:
from Helpers.data_cleaner import TweetProcessor

########################################## TRAIN ##################################################################
# Ensure the text data does not contain any NaN values
train_df_dirty['tweet'] = train_df_dirty['tweet'].fillna('')
train_texts = train_df_dirty['tweet'].tolist()  # Your training texts
train_labels = train_df_dirty['label'].tolist()  # Your training labels

# Instantiate and use the TweetProcessor
processor = TweetProcessor()
columns_to_clean = ['tweet']  # Add other columns if needed
cleaned_tweets_series_train = processor.process(train_df_dirty, columns=columns_to_clean)
cleaned_tweets_txt_train = processor.process(train_df_dirty, columns=columns_to_clean, output_format='txt')
cleaned_tweets_df_train = processor.process(train_df_dirty, columns=columns_to_clean, output_format='dataframe')

# Display results
#print(cleaned_tweets_series_train.head())
#print(cleaned_tweets_txt_train[:500])  # Display first 500 characters of cleaned text
print(cleaned_tweets_df_train)
cleaned_tweets_df_train.to_csv()
cleaned_tweets_df_train.to_csv('./DataSet/cleaned_tweets_train.csv')

######################################## TEST #####################################################################
# Ensure the text data does not contain any NaN values
#test_df_dirty['tweet'] = test_df_dirty['tweet'].fillna('')
test_texts = test_df_dirty['tweet'].tolist()  # Your training texts
#test_labels = test_df_dirty['label'].tolist()  # Your training labels

# Instantiate and use the TweetProcessor
processor = TweetProcessor()
columns_to_clean = ['tweet']  # Add other columns if needed
cleaned_tweets_series_test = processor.process(test_df_dirty, columns=columns_to_clean)
cleaned_tweets_txt_test = processor.process(test_df_dirty, columns=columns_to_clean, output_format='txt')
cleaned_tweets_df_test = processor.process(test_df_dirty, columns=columns_to_clean, output_format='dataframe')

# Display results
#print(cleaned_tweets_series_test.head())
#print(cleaned_tweets_txt_train[:500])  # Display first 500 characters of cleaned text
print(cleaned_tweets_df_test)
cleaned_tweets_df_train.to_csv()
cleaned_tweets_df_train.to_csv('./DataSet/cleaned_tweets_test.csv')

       label                                              tweet
id                                                             
0          1                             youre a complete penis
1          0                                   singing makes me
2          1            youre a waste of resources shaved pussy
3          0  facce da set thereaping thereapingtheseries co...
4          1           youre a total waste of everything lolita
...      ...                                                ...
44745      0  how is this parking by one member of your staf...
44746      1                            what a loser poop chute
44747      1  i talk to any girl who paid over 300 for her s...
44748      0  this makes me incredibly happy oitnb netflix l...
44749      1                  no one would miss you spread legs

[42318 rows x 2 columns]
                                                   tweet
id                                                      
31963  studiolife aislife re